In [6]:
import pandas as pd
import os

import numpy as np

from sklearn.datasets import fetch_california_housing

from evidently import ColumnMapping

from evidently.report import Report
from evidently.metrics.base_metric import generate_column_metrics
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset, DataQualityPreset
from evidently.metrics import *

from evidently.test_suite import TestSuite;
from evidently.tests.base_test import generate_column_tests
from evidently.test_preset import DataStabilityTestPreset, NoTargetPerformanceTestPreset
from evidently.tests import *

In [61]:
DIR_NAME = '/home/duclh3/Workspace/test_phase1_prob2/'

In [62]:
files = os.listdir(DIR_NAME)

In [63]:
df = pd.read_parquet("/mnt/f/Data/MLOPS_2023/data_phase-1/phase-1/prob-2/raw_train.parquet")


In [64]:
from src.data_processor.phase_1.prob1 import Phase1Prob1FeatureProcessorV1
from src.data_processor.phase_1.prob2 import Phase1Prob2FeatureProcessorV1
from src.model_predictor import Phase1Prob1ModelPredictor, Phase1Prob2ModelPredictor

In [65]:
DEFAULT_CONFIG_CHECKPOINTS = '../checkpoints'

phase1_prob1_pretrained_model = Phase1Prob1ModelPredictor(DEFAULT_CONFIG_CHECKPOINTS + '/phase-1/prob-1/v1.pkl')
phase1_prob1_feature_processor = Phase1Prob1FeatureProcessorV1()
phase1_prob2_pretrained_model = Phase1Prob2ModelPredictor(DEFAULT_CONFIG_CHECKPOINTS + '/phase-1/prob-2/v1.pkl')
phase1_prob2_feature_processor = Phase1Prob2FeatureProcessorV1()

In [67]:
for i in range(len(files)):
    print(i)
    test = pd.read_csv(DIR_NAME+'/'+files[i])
    data = phase1_prob2_feature_processor.process(test)
    prediction = phase1_prob2_pretrained_model.predict_proba(data)
    test['label'] = [1 if i >= 0.48 else 0 for i in prediction]
    report = Report(metrics=[
        DataDriftPreset(),
    ])
    report.run(reference_data=df, current_data=test)
    label_score = report.as_dict()['metrics'][1]['result']['drift_by_columns']['label']
    report.save_html(f'../analyst/phase-1/prob-2/file_{i}.html')

    print(label_score['drift_score'],label_score['drift_detected'])
    print(test['label'].value_counts(normalize=True))

0
0.018573234177763343 False
label
0    0.672
1    0.328
Name: proportion, dtype: float64
1
0.032774651282089846 False
label
0    0.653
1    0.347
Name: proportion, dtype: float64
2
0.005683516686290919 False
label
0    0.689
1    0.311
Name: proportion, dtype: float64
3
0.005683516686290919 False
label
0    0.689
1    0.311
Name: proportion, dtype: float64
4
0.026072881307957936 False
label
0    0.662
1    0.338
Name: proportion, dtype: float64
5
0.0010885896415525618 False
label
0    0.695
1    0.305
Name: proportion, dtype: float64
6
0.0140448735051809 False
label
0    0.678
1    0.322
Name: proportion, dtype: float64
7
0.040171493831861346 False
label
0    0.643
1    0.357
Name: proportion, dtype: float64
8
0.019325832505604053 False
label
0    0.671
1    0.329
Name: proportion, dtype: float64
9
0.019325832505604053 False
label
0    0.671
1    0.329
Name: proportion, dtype: float64
10
0.014801142516888157 False
label
0    0.677
1    0.323
Name: proportion, dtype: float64
11
0.00797

IndexError: list index out of range

In [38]:
i=99
report.run(reference_data=df, current_data=test)
report.save_html(f'../analyst/phase-1/prob-1/file_{i}.html')

In [43]:
test['label'].value_counts(normalize=True)

label
0    0.963
1    0.037
Name: proportion, dtype: float64

In [46]:
df['label'].value_counts(normalize=True)

label
0    0.936685
1    0.063315
Name: proportion, dtype: float64

In [58]:
label_score = report.as_dict()['metrics'][1]['result']['drift_by_columns']['label']
print(label_score['drift_score'],label_score['drift_detected'])

0.04286388547269783 False


0      0
1      0
2      0
3      0
4      0
      ..
995    0
996    0
997    0
998    0
999    0
Name: label, Length: 1000, dtype: int64